In [ ]:
import sys
import pickle
import warnings
import pandas as pd
from collections import defaultdict

warnings.filterwarnings("ignore")

In [ ]:
sys.path[1:1] = ["/home/ec2-user/SageMaker/GitHub Repos/cipy/cipy/"]
sys.path[2:2] = ["/home/ec2-user/SageMaker/Users/SP056963/work_queue_prioritization_v2/"]
sys.path[3:3] = ["/home/ec2-user/SageMaker/Users/SP056963/work_queue_prioritization_v2/02_feat_sel_and_imp/"]
sys.path[4:4] = ["/home/ec2-user/SageMaker/Users/SP056963/work_queue_prioritization_v2/02_feat_sel_and_imp/professional_claims/02_step/"]

sys

In [ ]:
from wqp_methods import *

In [ ]:
# Reading the Institutional claims dataset
ref_col= "claim_filing_ind_code3"

# csv_path for Institutional Claims Data Path
csv_path = "/home/ec2-user/SageMaker/Users/SP056963/work_queue_prioritization_v2/01_data_preprocessing/preprocessed_datasets/"

prof_claims= pd.read_csv(csv_path+"pc_preprocessed_dataset_2021-06-01.csv")

# Creating the event_flag - for survival analysis.
prof_claims["event_flag"] = True

In [ ]:
claim_filing_ind_codes = inst_claims[ref_col].unique()

In [ ]:
rfe_feat_file = "pc_features_for_rfe.pickle"

with open(rfe_feat_file, "rb") as input_file:
    rfe_feat_op = pickle.load(input_file)

In [ ]:
cfic_tracker = {}

for claim_filing_ind_code in claim_filing_ind_codes:
    
    print("Claim Filing Indicator Code: {}".format(claim_filing_ind_code))
    
    cfic_df = inst_claims[inst_claims[ref_col]==claim_filing_ind_code].reset_index(drop=True)

    folds = get_folds(cfic_df)
    
    cfic=""
    if "medicaid" in claim_filing_ind_code.lower():
        cfic = "medicaid"
    elif "medicare" in claim_filing_ind_code.lower():
        cfic = "medicare"
    else:
        cfic = "commercial"
        
    num_feats = ic_features_for_rfe[cfic]["num_feat"]
    cat_feats = ic_features_for_rfe[cfic]["cat_feat"]

    encoder_dict = defaultdict(LabelEncoder)
    cat_df = cfic_df[cat_feats].astype("str")
    cat_df = cat_df.apply(lambda x: encoder_dict[x.name].fit_transform(x))

    num_df = cfic_df[num_feats]

    res_df = cfic_df[["response_time", "event_flag"]]

    rsf_df = pd.concat([num_df, cat_df, res_df], axis=1)
    
    features_tracker, c_index_tracker, feat_imp_tracker = recursive_feature_elimination(folds, rsf_df, num_feats+cat_feats)
    
    cfic_tracker[claim_filing_ind_code] = (features_tracker, c_index_tracker, feat_imp_tracker)

In [ ]:
pickle.dump(cfic_tracker, open("ic_rfe_output.pickle", "wb"))